# Import Statements #

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import ipywidgets as widgets
import trimesh as tm
import scipy as sc
import numpy as np
import pandas as pd
from itertools import cycle
import skcriteria as sk
from skcriteria.agg import similarity  # here lives TOPSIS
from skcriteria.pipeline import mkpipe  # this function is for create pipelines
from skcriteria.preprocessing import invert_objectives, scalers
pv.set_jupyter_backend('trame')

#Display full array without truncation
np.set_printoptions(threshold=np.inf)

## 01. Loading the base object ##

We are loading the base .obj file here to create voxels. The voxel size is considered here is 3x3x3m and the base object is extruded to 2m so that a grid of only 1 voxel in depth is created. In the Hub implementation we can consider adding code to create a 2d voxel grid with the current 3d implementation

In [2]:
vs = 3
unit = [vs, vs, vs]
tol = 1e-09
mesh_path = os.path.relpath(r"C:\Users\aditya.soman\source\repos\Loci_Python\Obj_Data\floorobject.obj")
mesh = tg.geometry.load_mesh(mesh_path)


In [3]:
sample_cloud, ray_origins = tg.geometry.mesh_sampling(mesh, unit, multi_core_process=False, return_ray_origin = True, tol=tol)
lattice = sample_cloud.voxelate(unit, closed=True)

In [4]:
# initiating the plotter
p = pv.Plotter(notebook=True) # ITK plotter for interactivity within the python notebook (itkwidgets library is required)


# fast visualization of the point cloud
#sample_cloud.fast_notebook_vis(p)

# fast visualization of the lattice
lattice.fast_notebook_vis(p)

# adding the base mesh: light blue
mesh = pv.read(mesh_path)
#p.add_mesh(mesh, color='#abd8ff')

# adding the ray origins: dark blue
#p.add_points(pv.PolyData(ray_origins), color='#004887')

# plotting
p.show()


Widget(value='<iframe src="http://localhost:62141/index.html?ui=P_0x13846fa5390_0&reconnect=auto" class="pyvis…

## 02. Create Various Performance matrices (Lattices in topogenesis terminology)
1. Quiteness : I have added a few points in Rhino as potential noise sources and the argmin distances from these points are considered as the level of quiteness for that voxel. Note quiteness is used and not dB levels since we are not calculating any sound insulations etc.
2. Distance from Facades : Distance from facades N,S,E,W. If the users have a priority to configure zones close to a specific facade 
3. Daylight : Simplified daylight analysis values for the voxels 
4. Distance from Lobby : It is the distance of the voxels from the Lobby entrances 
5. Distance from an Core: It is the distance of the voxels from the Core entrances 

In [5]:
class PerformanceLattice:
    def __init__(self, base_lattice):
        self.base_lattice = base_lattice
        self.north_distance = np.max(base_lattice.centroids.T[1])
        self.south_distance = np.min(base_lattice.centroids.T[1])
        self.east_distance = np.max(base_lattice.centroids.T[0])
        self.west_distance = np.min(base_lattice.centroids.T[0])

    def create_distance_lattice(self, point_lattice:np.array) -> None:
        """
        Create a distance based lattice  on the point sources and initial lattice.
        The resulting distance is arg min values from the closest point
        """
        # Create availability lattice
        init_lattice = self.base_lattice
        availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
        voxel_coordinates = availability_lattice_voxels.centroids

        # Flatten the initial lattice
        flattened_lattice = self.base_lattice.flatten()

        # Compute Euclidean distances
        eucledian_distance = sc.spatial.distance.cdist(point_lattice, voxel_coordinates)
        distance_from_each_source = eucledian_distance.T

        # Compute the average quiteness values
        average_distance_indexing = np.argmin(distance_from_each_source, axis=1)
        average_distance_values = [branch[index] for branch, index in zip(distance_from_each_source, average_distance_indexing)]

        # Compute the quiteness lattice
        distance_lattice_padded = np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(average_distance_values))])
        distance_lattice_np = distance_lattice_padded.reshape(self.base_lattice.shape)

        # Convert to lattice format
        distance_lattice = tg.to_lattice(distance_lattice_np, distance_lattice_np.shape)

        return distance_lattice
    
    def create_facade_lattice(self, direction:str) -> None:
        if direction == 'north':
            evaluation_value = self.north_distance
            distance_lattice = evaluation_value- self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape)
        elif direction == 'south':
            evaluation_value = self.south_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape))
        elif direction == 'east':
            evaluation_value = self.east_distance
            distance_lattice = evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape)
        elif direction == 'west':
            evaluation_value = self.west_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape))
        return distance_lattice
    
    def distance_from_outermostvoxel (self):
        return self.base_lattice

In [6]:
def csv_nparray(file_name:str) -> np.array:
    """
    Convert a csv file to numpy array
    """
    Complete_file = pd.read_excel(file_name, sheet_name=0,engine='openpyxl',header = None )
    array_excel = Complete_file.to_numpy()
    point_array = array_excel.T
    return point_array
# Numpy array of points indicating source of noise
Noise_sources = csv_nparray('Noise_sources.xlsx')
Core_centers = csv_nparray('Core_Centers.xlsx')

# Create Performance Lattice creator class
Performance_Creator = PerformanceLattice(lattice+1)
Quiteness_lattice = Performance_Creator.create_distance_lattice(Noise_sources)
Coredis_lattice = Performance_Creator.create_distance_lattice(Core_centers)
East_facade_lattice = Performance_Creator.create_facade_lattice('east')
West_facade_lattice = Performance_Creator.create_facade_lattice('west')
North_facade_lattice = Performance_Creator.create_facade_lattice('north')
South_facade_lattice = Performance_Creator.create_facade_lattice('south')
#Closeness to Facade can be equated to daylight availability
Daylight_simplified = Performance_Creator.distance_from_outermostvoxel()


In [7]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data Select whichever analysis you want to display
grid.cell_data["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
# Add your points to plot 
p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)

p.show()

Widget(value='<iframe src="http://localhost:62141/index.html?ui=P_0x13846f108d0_1&reconnect=auto" class="pyvis…

## 03. Create Desirability lattices ( Perform MCDA on the  based on the various performance matrices developed in the previous step ) ##

In [8]:
# Collect all the Performance matrice
array_all_performance_matrices = [Quiteness_lattice.flatten(),Coredis_lattice.flatten(), East_facade_lattice.flatten(), West_facade_lattice.flatten(), North_facade_lattice.flatten(), South_facade_lattice.flatten()]
performance_matrix = np.array(array_all_performance_matrices).T

In [9]:
class DesirabilityLattice:
    def __init__(self, performance_matrix:np.array):
        self.performance_matrix = performance_matrix
    
    def topsis (self, objectives_array, weights_array, criteria_array):
        """
        TOPSIS method to calculate the desirability of each voxel
        """
        decision_matrix = sk.mkdm(
        self.performance_matrix,
        objectives = objectives_array,
        weights = weights_array,
        criteria = criteria_array)

        pipe = mkpipe(
        invert_objectives.NegateMinimize(),
        scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix
        scalers.SumScaler(target="weights"),  # and this transform the weights
        similarity.TOPSIS(),
        )
        rank = pipe.evaluate(decision_matrix)
        mcdm_result= rank.e_.similarity.reshape(lattice.shape)
        
        return mcdm_result


In [10]:
criteria_array = ["quietness", "coredis", "eastfacade", "westfacade", "northfacade", "southfacade"]
objectives_array = [max,min,max,max,max,max]
decisionmaker = DesirabilityLattice(performance_matrix)

#Zone1 parameters
Z1_weights_array = [0.5, 0.5, 0, 0, 0.6,0]
Z1_Desirability_lattice = decisionmaker.topsis(objectives_array, Z1_weights_array, criteria_array)

#Zone2 parameters
Z2_weights_array = [0.5, 0.5, 0, 0, 0.6,0]
Z2_Desirability_lattice = decisionmaker.topsis(objectives_array, Z2_weights_array, criteria_array)

#Zone3 parameters
Z3_weights_array = [0.5, 0.5, 0, 0, 0.6,0]
Z3_Desirability_lattice = decisionmaker.topsis(objectives_array, Z3_weights_array, criteria_array)

In [11]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data Select whichever analysis you want to display
grid.cell_data["Zone 1 Desirability Lattice"] = Z1_Desirability_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes
# Matplot lib colormaps https://matplotlib.org/stable/users/explain/colors/colormaps.html

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="BrBG", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
# Add your points to plot 
p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)

p.show()

Widget(value='<iframe src="http://localhost:62141/index.html?ui=P_0x138548984d0_2&reconnect=auto" class="pyvis…

## 04. Initiate Agent classes ##

The Computational agents have certain behaviours and can take certain topological actions in the program. In the case of this notebook the agents can occupy ( claim) voxels in the floor plan for a particular zone during an instance of a simulation. The agents can navigate through the voxelated space by means of a stencil which defines the neighbourhood definition like Von-neumann neighbourhood or Moora neighbourhood (https://en.wikipedia.org/wiki/Von_Neumann_neighborhood#:~:text=The%20von%20Neumann%20neighbourhood%20of,cellular%20automaton%20in%20three%20dimensions.)

In [ ]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 


In [46]:
# Definition for an agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()  

In [47]:
# Definition for an enviornment class
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()

    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

## 05. Run Agent Based Simulation ##

In [48]:
available_index_list= list(np.argwhere(lattice.flatten()==1).flatten())
# Defining agents
def initialize_agents_random_origin (stencil,avail_lattice,agn_num):
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), 1)
    agn_origins = np.unravel_index(select_id,avail_lattice.shape) 
    myagent = agent(agn_origins, stencil, agn_num)
    return myagent

Agent_Z1=initialize_agents_random_origin (stencil_squareness,lattice*1,9)  
avail_lattice = lattice*1 

In [51]:
occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
env_P_H = {"availibility": occ_lattice_sim,"enviornment": Z1_Desirability_lattice}
env_1 = environment(env_P_H, Agent_Z1,40,"one_neighbour_occupy_squareness_moore")

In [52]:
env_availability_viz = []

env_list =[env_1]
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
#print(number_steps),env_2,env_3,env_4,env_5,env_6,env_7
    
for a in range(number_steps):
    # print(env.availibility)
    # print(env.agent_origin)

    for e in env_list:
        if a < e.number_of_iterations :
            print(a)
            print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            env_availability_viz.append(e.availibility-1)

0
40


TypeError: only integer scalar arrays can be converted to a scalar index

In [37]:
greater_than_2 = env_availability_viz[0][env_availability_viz[0] > 2]
greater_than_2

lattice([8])

In [38]:
p = pv.Plotter(notebook=True)

base_lattice = avail_lattice

# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = base_lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = base_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
base_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 


# Add the data values to the cell data
grid.cell_data["Agents"] = env_availability_viz[0].flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([1, avail_lattice.size])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1, show_scalar_bar=False)


p.show()

Widget(value='<iframe src="http://localhost:62141/index.html?ui=P_0x1382e1b84d0_7&reconnect=auto" class="pyvis…